# MS-E2121 Pretask: checking your Julia installation
This notebook implements the paint factory problem from the first lecture. It uses all the Julia packages that we will be using in the first weeks. If you can run this without errors, you should be good to go. You can also use this code as learning material if you wish.

In [ ]:
# Import necessary packages
using JuMP, Cbc, Plots

## Input data
Formatting the data we will be using in the model

In [ ]:
# Raw materials and their daily availability
materials = ["M1" "M2"]
A = [24 6]
# Products and their profits
products = ["Exterior" "Interior"]
P = [5 4]
# Material requirements
R = [6 4; 1 2]

In [ ]:
# Converting arrays to dictionaries
# This is not strictly necessary, but makes building the model easier
# because the labels "M1", "M2" etc. can be used to refer to the elements instead of the array indices
a_dict = Dict(materials .=> A)
p_dict = Dict(products .=> P)
r_dict = Dict( (materials[i], products[j]) => R[i,j]
          for i in 1:length(materials), j in 1:length(products) )

## Building the model
We start with an empty JuMP (Julia for Mathematical Programming) model, then add the decision variables, objective function and constraints.

In [ ]:
# Start building an optimization model named "paint"
paint = Model(Cbc.Optimizer)

In [ ]:
# Decision variables: the production amount for each product
# We definitely want this to be nonnegative, negative production wouldn't make sense
@variable(paint, x[products] >= 0)

In [ ]:
# Objective function: maximize total profit from selling the products
@objective(paint, Max, sum(p_dict[j]*x[j] for j in products))

In [ ]:
# Constraints as specified on the lecture slides.
# Note that constraints do not need a name, it can be omitted.
# However, naming the constraints makes it easier to debug your model
@constraint(paint, supply[i in materials], sum(r_dict[i,j]*x[j] for j in products) <= a_dict[i] )
@constraint(paint, int_demand, sum(x["Interior"]) <= 2)
@constraint(paint, prod_balance, x["Interior"] <= x["Exterior"] + 1)

## Checking the model and obtaining solutions
We check that the printed model matches what we have in the lecture material and solve it to optimality.

In [ ]:
# Print the model and optimize it
println(paint)
optimize!(paint)

In [ ]:
# Extract results and print solution
obj_val = round(objective_value(paint), digits=1)
ext_val = round(value(x["Exterior"]), digits=1)
int_val = round(value(x["Interior"]), digits=1)
println("The maximum profit is $obj_val when $ext_val tons of exterior and $int_val tons of interior paint are produced.")

## Making a plot of the problem
There is a lot of code here, you don't need to understand it now, but you can come back to it later if you need to make plots yourself.

In [ ]:
# Some ranges to help with plotting, these are similar to what you may have used in Matlab
ext_range = collect(0:0.1:5)
int_range = collect(0:0.1:5)

In [ ]:
# Plot the constraints. An exclamation mark after a function name usually means that it mutates
# its arguments, here the plot! function adds to the existing plot instead of creating a new one
plot(ext_range, (-6*ext_range .+ 24)./4, xlim=(0,5), ylim=(0,5), xlabel="exterior", ylabel="interior", label="6x1+4x2=24")
plot!(ext_range, (-ext_range .+ 6)./2, label="x1+2x2=6")
plot!(ext_range, (ext_range.+1), label="x2-x1=1")
hline!([2], label="x2=2")

In [ ]:
# A simple way to plot the feasible region, loop through a grid of points and
# only plot those that satisfy all constraints.
A = [6 4; 1 2; -1 1; 0 1]
b = [24; 6; 1; 2]
feasible_region = []
for x1 in ext_range
    for x2 in int_range
        if all(A*[x1; x2] .<= b)
            push!(feasible_region, [x1, x2])
        end
    end
end

In [ ]:
# Turn the array of arrays into a 2D-array and plot the points
feasible_region = hcat(feasible_region...)
scatter!(feasible_region[1,:], feasible_region[2,:], ms=1, label=false)

In [ ]:
# Plot the objective function at the optimal value (21) obtained earlier
plot!(ext_range, (-5*ext_range.+21)./4, linestyle=:dash, linewidth=2, color=:black, label="Objective function")